In [2]:
from osgeo import gdal      # Necessary to do this import to get rasterio to import
import rasterio
import numpy as np
import time

In [5]:
# General paths and constants

general_uri = 's3://gfw2-data/forest_change/GLAD_Europe_height_data/'

local_out_dir = 'C:\\GIS\\Carbon_model_Europe\\outputs\\'

timestr = time.strftime("%Y%m%d")

tile_size = 10      # Tile size is from the top left of the tile

In [ ]:
# To make 10x10 tiles:
# gdalwarp from subprocess.check_call(cmd) isn't working
# cmd = ['gdalwarp', '-tr', '0.00025', '0.00025', '-co', 'COMPRESS=DEFLATE', '-tap', '-te', str(10), str(49), str(11), str(50), '-dstnodata', '0', '-t_srs', 'EPSG:4326', 
#        '-overwrite', '-progress', '/vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2021.tif', 'C:\\GIS\\Carbon_model_Europe\\outputs\\50N_010E_FH_2021.tif']
# check_call(cmd)
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2021.tif 50N_010E_FH_2021.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2020.tif 50N_010E_FH_2020.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2019.tif 50N_010E_FH_2019.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/FH_2018.tif 50N_010E_1FH_2018.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/DFL_2021.tif 50N_010E_DFL_2021.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/DFL_2020.tif 50N_010E_DFL_2020.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/DFL_2019.tif 50N_010E_DFL_2019.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/forest_change/GLAD_Europe_height_data/202307_revision/DFL_2018.tif 50N_010E_DFL_2018.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/50N_010E_tree_cover_loss_driver_processed.tif 50N_010E_1deg_tree_cover_loss_driver_processed.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_fires/20230315/processed/50N_010E_tree_cover_loss_fire_processed.tif 50N_010E_1deg_tree_cover_loss_fire_processed.tif
# gdalwarp -tr 0.00025 0.00025 -co COMPRESS=DEFLATE -tap -te 10 40 20 50 -dstnodata 0 -t_srs EPSG:4326 -overwrite /vsis3/gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/50N_010E_peat_mask_processed.tif 50N_010E_1deg_peat_mask_processed.tif


In [10]:
# To make dummy forest height 10x10 tiles with random integers. Need to change the new_raster_path to make different years. 
# existing_raster_path can be reused for different years

existing_raster_path = forest_height_previous_uri = f'{general_uri}202307_revision/test_10x10_deg/50N_010E_FH_2020.tif'

new_raster_path = f'{local_out_dir}50N_010E_FH_2017_random_data.tif'

with rasterio.open(existing_raster_path) as src:

    profile = src.profile
       
    data = np.random.randint(0, 27, (profile['height'], profile['width']))

    # # Create the new raster file using the metadata from the existing raster
    with rasterio.open(new_raster_path, 'w', **profile) as dst:
        dst.write(data, 1)  # Write the empty array to the new raster

In [10]:
# To make dummy other input 10x10 tiles with random integers

# existing_raster_path = f'{general_uri}202307_revision/test_10x10_deg/50N_010E_DFL_2018.tif'
# new_raster_path = f'{local_out_dir}50N_010E_DFL_2018_random_data.tif'

# existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/50N_010E_tree_cover_loss_driver_processed.tif"
# new_raster_path = f'{local_out_dir}50N_010E_tree_cover_loss_driver_processed_random_data.tif'

# existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/planted_forest_type/SDPT_v1/standard/20200730/50N_010E_plantation_type_oilpalm_woodfiber_other_unmasked.tif"
# new_raster_path = f'{local_out_dir}50N_010E_plantation_type_oilpalm_woodfiber_other_unmasked_random_data.tif'

# existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/50N_010E_peat_mask_processed.tif"
# new_raster_path = f'{local_out_dir}50N_010E_peat_mask_processed_random_data.tif'

# existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_fires/20230315/processed/50N_010E_tree_cover_loss_fire_processed.tif"
# new_raster_path = f'{local_out_dir}50N_010E_tree_cover_loss_fire_processed_random_data.tif'

existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_2021_50N_010E.tif"
new_raster_path = f'{local_out_dir}50N_010E_burned_area_2017_random_data.tif'

with rasterio.open(existing_raster_path) as src:

    profile = src.profile
       
    # data = np.random.randint(0, 2, (profile['height'], profile['width']))   # existing_raster_path = f'{general_uri}202307_revision/test_10x10_deg/50N_010E_DFL_2020.tif'
    # data = np.random.randint(0, 6, (profile['height'], profile['width']))   # existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_drivers/processed/drivers_2022/20230407/50N_010E_tree_cover_loss_driver_processed.tif"
    # data = np.random.randint(0, 4, (profile['height'], profile['width']))   # existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/planted_forest_type/SDPT_v1/standard/20200730/50N_010E_plantation_type_oilpalm_woodfiber_other_unmasked.tif"
    # data = np.random.randint(0, 2, (profile['height'], profile['width']))   # existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/50N_010E_peat_mask_processed.tif"
    # data = np.random.randint(0, 22, (profile['height'], profile['width']))  # existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/tree_cover_loss_fires/20230315/processed/50N_010E_tree_cover_loss_fire_processed.tif"
    data = np.random.randint(0, 2, (profile['height'], profile['width']))   # existing_raster_path = "s3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_2021_50N_010E.tif"
    
    # # Create the new raster file using the metadata from the existing raster
    with rasterio.open(new_raster_path, 'w', **profile) as dst:
        dst.write(data, 1)  # Write the empty array to the new raster